##### <img src="mioti.png" style="height: 100px">

# Challenge S5: Modelos regresivos aplicados a series temporales

## Objetivo

El objetivo de este challenge es usar nuestro dataframe de ventas en retail y hacer un modelo para la predicción de ventas para un artículo distinto del que vimos en el worksheet.

Tendréis que explorar los datos para ver el comportamiento del artículo elegido y las peculiaridades que pueda tener.

Luego tendréis que hacer un modelo que pueda predecir las ventas de este artículo usando todo lo aprendido anteriormente, probando distintos modelos, hiperparametrizándolos y por último añadiendo todas las variables sintéticas que veáis convenientes para que el modelo aprenda mejor del comportamiento de las unidades vendidas del producto.

## 1. Configuración del entorno

Cargamos los paquetes que vamos a utilizar en la sesión.

In [21]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

## 2. Carga de los datos, filtrado de nuestro artículo y visualización de los datos
 



**Carga el dataframe**

In [22]:
dataRetail = pd.read_csv('Dataframe_Retail.csv', delimiter=',')
print(dataRetail['Referencia'].unique())

['Cerveza tercio' 'Baguetes 2 unidades' 'Aceite de oliva virgen extra 5 l'
 'Bolsa naranjas 2Kg.' 'Mandarina' 'Melo  OR' 'Jamón reserva'
 'Naranja en cajas' 'Pack de plátanos'
 'MERLUZA PALANGRE 1 - 2 Kg (1 Caja 10Q.)' 'Tomate jugoso al peso'
 'Filetes de salmón' 'Sepia' 'Fresa premium 500gr']


**Usa una máscara para filtrar el artículo sobre el que queremos realizar las predicciones**

In [23]:
dataSku = dataRetail[dataRetail['Referencia']=='Jamón reserva']
dataSku['Fecha'] = pd.to_datetime(dataSku['Fecha'], format='%Y-%m-%d')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


** Ya que hemos cogido un producto que tiene huecos vamos a hacer una limpieza del dataframe ya que esos huecos originarían errores más tarde**

In [24]:
dataSku=dataSku.set_index('Fecha')

In [25]:
fechaMin = dataSku.index.min()
print(fechaMin)

2017-01-02 00:00:00


In [26]:
fechaMax = dataSku.index.max()
print(fechaMax)

2019-05-18 00:00:00


In [27]:
range = pd.date_range(start=fechaMin, end=fechaMax,freq='d')
dataSku = dataSku.reindex(range)
dataSku['Unidades']=dataSku['Unidades'].fillna(value=0)
dataSku['Ventas']=dataSku['Ventas'].fillna(value=0)
dataSku['Referencia']=dataSku['Referencia'].fillna(method='ffill')
dataSku=dataSku.reset_index()
dataSku=dataSku.rename(columns={'index':'Fecha Format'})
#dataSku['OffsetDays']=dataSku['OffsetDays'].astype(int)
dataSku['OffsetDays'] = dataSku.index

In [28]:
dataSku.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Fecha Format  867 non-null    datetime64[ns]
 1   Referencia    867 non-null    object        
 2   Unidades      867 non-null    float64       
 3   Ventas        867 non-null    float64       
 4   OffsetDays    867 non-null    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 34.0+ KB


In [29]:
dataSku.tail(30)

,Fecha Format,Referencia,Unidades,Ventas,OffsetDays
837,2019-04-19,Jamón reserva,2.355,44.490000,837
838,2019-04-20,Jamón reserva,110.885,1952.279999,838
839,2019-04-21,Jamón reserva,2.915,55.059999,839
840,2019-04-22,Jamón reserva,1.675,31.630000,840
841,2019-04-23,Jamón reserva,108.500,2001.369996,841
842,2019-04-24,Jamón reserva,76.760,1080.600003,842
843,2019-04-25,Jamón reserva,55.790,995.920005,843
844,2019-04-26,Jamón reserva,69.270,1300.410007,844
845,2019-04-27,Jamón reserva,76.545,1474.170007,845
846,2019-04-28,Jamón reserva,0.520,11.440000,846


In [30]:
dataSku[dataSku['Unidades']==0]

,Fecha Format,Referencia,Unidades,Ventas,OffsetDays
4,2017-01-06,Jamón reserva,0.0,0.0,4
20,2017-01-22,Jamón reserva,0.0,0.0,20
69,2017-03-12,Jamón reserva,0.0,0.0,69
83,2017-03-26,Jamón reserva,0.0,0.0,83
102,2017-04-14,Jamón reserva,0.0,0.0,102
119,2017-05-01,Jamón reserva,0.0,0.0,119
274,2017-10-03,Jamón reserva,0.0,0.0,274
314,2017-11-12,Jamón reserva,0.0,0.0,314
321,2017-11-19,Jamón reserva,0.0,0.0,321
335,2017-12-03,Jamón reserva,0.0,0.0,335


In [31]:
dataSku.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Fecha Format  867 non-null    datetime64[ns]
 1   Referencia    867 non-null    object        
 2   Unidades      867 non-null    float64       
 3   Ventas        867 non-null    float64       
 4   OffsetDays    867 non-null    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 34.0+ KB


## 3. Prueba de la predicción sobre nuestras unidades vendidas con distintos modelos

** Con la función de evaluación puedes probar el funcionamiento de distintos modelos**

In [32]:
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))
def calculoEvaluacionModelo(modelo,df,norm):

    xPredicted = []
    yPredicted = []
    ySpotted = []
    mean_error = []

    for day in range(800,861):
        train = df[df['OffsetDays'] < day]
        val = df[df['OffsetDays'] == day]       
        
        xtr, xts = train.drop(['Unidades'], axis=1), val.drop(['Unidades'], axis=1)
        
        if(norm==True):
            sc = StandardScaler()  
            xtr = sc.fit_transform(xtr)  
            xts = sc.transform(xts)  
        ytr, yts = train['Unidades'].values, val['Unidades'].values  

        mdl = modelo
        mdl.fit(xtr, ytr)

        p = mdl.predict(xts)

        xPredicted=xPredicted + [day]
        yPredicted=yPredicted + [p[0]]
        ySpotted = ySpotted + [yts[0]]

        error = rmsle(yts, p)
        print('Month %d - Error %.5f' % (day, error))
        mean_error.append(error)

    print('Mean Error = %.5f' % np.mean(mean_error))
    prediction = pd.DataFrame(
        {'Sold units predicted': yPredicted,
         'Sold units actually spotted': ySpotted
        },index=xPredicted)
    prediction.plot()

** Elige el modelo que más te haya gustado o mejor haya funcionado e hiperparametrizalo si quieres**

In [33]:
norma=False
dataSku_simple = dataSku[['OffsetDays','Unidades']].copy()
dataSku_simple.info()
calculoEvaluacionModelo(RandomForestRegressor(n_estimators=20, random_state=0),dataSku_simple,norma)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   OffsetDays  867 non-null    int64  
 1   Unidades    867 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 13.7 KB


TypeError: 'DatetimeIndex' object is not callable

In [ ]:
dataSku_simple.info()

## 4. Complementar el dataframe con variables sintéticas

** En base a lo que hayas observado en el comportamiento de las unidades vendidas de tu referencia crea variables sintéticas que creas que ayuden al modelo a mejorar la predicción**